In [ ]:
import nest_asyncio
nest_asyncio.apply()  # Enable async in Jupyter

import asyncio
import aiohttp
import pandas as pd
import hashlib
from datetime import datetime, timedelta
import os
import pytz
import holidays
import random
import json
import shutil
import logging # Import logging module

# Memory threshold (e.g., 90% of available memory)
MEMORY_THRESHOLD = 0.9


In [ ]:
# --- Logging Configuration ---
class HKTFormatter(logging.Formatter):
    def formatTime(self, record, datefmt=None):
        hkt_tz = pytz.timezone('Asia/Hong_Kong')
        dt = datetime.fromtimestamp(record.created, tz=pytz.UTC).astimezone(hkt_tz)
        return dt.strftime(datefmt or '%Y-%m-%d %H:%M:%S')

logger = logging.getLogger('NewsMonitor')
logger.handlers = []
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler('monitor.log', encoding='utf-8')
file_handler.setLevel(logging.INFO)
formatter = HKTFormatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

discord_logger = logging.getLogger('DiscordLogger')
discord_logger.handlers = []
discord_logger.setLevel(logging.INFO)
discord_file_handler = logging.FileHandler('discord_messages.log', encoding='utf-8')
discord_file_handler.setLevel(logging.INFO)
discord_formatter = HKTFormatter('%(asctime)s - News ID: %(news_id)s - Message: %(discord_message)s')
discord_file_handler.setFormatter(discord_formatter)
discord_logger.addHandler(discord_file_handler)

health_check_logger = logging.getLogger('HealthCheckLogger')
health_check_logger.handlers = []
health_check_logger.setLevel(logging.INFO)
health_check_file_handler = logging.FileHandler('health_check.log', encoding='utf-8')
health_check_file_handler.setLevel(logging.INFO)
health_check_formatter = HKTFormatter('%(asctime)s - %(message)s')
health_check_file_handler.setFormatter(health_check_formatter)
health_check_logger.addHandler(health_check_file_handler)
# --- End Logging Configuration ---

# Configuration
FINNHUB_API_KEY = "cumn7d1r01qsapi0gk1gcumn7d1r01qsapi0gk20"  # Confirmed valid
DISCORD_WEBHOOK = "https://discord.com/api/webhooks/1389596344674680893/h4vaSPCE2I0HyolHgp_EM-b1CIiGwZrm19B7qBLEPd3eHhqYm-J9DfgsT-dHFZkx8PCa"  # Replace with valid webhook
DISCORD_ALERTS = True
NEWS_STORAGE_DIR = "news_data"
METADATA_FILE = os.path.join(NEWS_STORAGE_DIR, "latest_news_timestamps.json")
TICKERS = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "NVDA", "JPM", "JNJ", "V", "PG",
    "HD", "MA", "DIS", "PYPL", "BAC", "NFLX", "ADBE", "CRM", "KO", "PEP",
    "TMO", "ABT", "AVGO", "CSCO", "CMCSA", "XOM", "WMT", "VZ", "MRK", "PFE",
    "INTC", "T", "ABBV", "ORCL", "CVX", "ACN", "DHR", "MCD", "NKE", "PM"
]
BATCH_SIZE = 10
os.makedirs(NEWS_STORAGE_DIR, exist_ok=True)

In [ ]:
class NewsFetcher:
    def __init__(self, api_key):
        self.api_key = api_key

    async def fetch_single_ticker_news(self, ticker, session):
        request_time = datetime.now(pytz.UTC).strftime('%Y-%m-%d %H:%M:%S UTC')
        current_date = datetime.now(pytz.UTC)
        from_date = (current_date - timedelta(days=1)).strftime('%Y-%m-%d')  # Yesterday
        to_date = current_date.strftime('%Y-%m-%d')  # Today
        url = f"https://finnhub.io/api/v1/company-news?symbol={ticker}&from={from_date}&to={to_date}&token={self.api_key}"
        logger.debug(f"Fetching news for {ticker} with URL: {url}")
        
        backoff = [2, 4, 8]
        for retry in range(3):
            try:
                async with session.get(url) as response:
                    response_time = datetime.now(pytz.UTC).strftime('%Y-%m-%d %H:%M:%S UTC')
                    if response.status == 429:
                        delay = backoff[retry] + random.uniform(0.1, 0.5)
                        logger.warning(f"Rate limit hit for {ticker}. Retrying in {delay:.2f}s... (Attempt {retry+1})")
                        await asyncio.sleep(delay)
                        continue
                    if response.status == 200:
                        news = await response.json()
                        logger.debug(f"Raw news for {ticker}: {news}")
                        logger.info(f"[API] Fetched {len(news)} items for {ticker} - Request: {request_time}, Response: {response_time}")
                        processed_news = []
                        for item in news:
                            if 'datetime' in item and isinstance(item['datetime'], (int, float)) and 'id' in item:
                                item_date = datetime.fromtimestamp(item['datetime'], tz=pytz.UTC)
                                # Filter strictly within yesterday to today
                                if from_date <= item_date.strftime('%Y-%m-%d') <= to_date:
                                    item['ticker'] = ticker
                                    item['summary'] = item.get('summary', '')
                                    processed_news.append(item)
                                else:
                                    logger.debug(f"Skipping item {item.get('id')} for {ticker}: Date {item_date.strftime('%Y-%m-%d')} outside {from_date} to {to_date}")
                            else:
                                logger.warning(f"[Warning] Skipping news item for {ticker} due to missing/invalid 'datetime' or 'id': {item.get('headline', 'N/A')}")
                        if not processed_news:
                            logger.info(f"[Info] No news within range for {ticker} at {response_time}")
                        return processed_news
                    else:
                        logger.error(f"[Error] HTTP {response.status} for {ticker} at {response_time}. (Attempt {retry+1})")
                        return []
            except aiohttp.ClientError as e:
                logger.error(f"[Error] Network error for {ticker} (retry {retry+1}): {e}")
                await asyncio.sleep(backoff[retry] + random.uniform(0.1, 0.5))
            except Exception as e:
                logger.error(f"[Error] Unexpected error for {ticker} (retry {retry+1}): {e}")
                await asyncio.sleep(backoff[retry] + random.uniform(0.1, 0.5))
        logger.error(f"[Error] Failed to fetch news for {ticker} after 3 retries.")
        return []

    async def fetch_batch(self, tickers, session):
        all_news = []
        for ticker in tickers:
            await asyncio.sleep(0.1)  # 0.1s delay between calls
            result = await self.fetch_single_ticker_news(ticker, session)
            if isinstance(result, list):
                all_news.extend(result)
            else:
                logger.error(f"[Error] Failed to fetch news for {ticker}: {result}")
        return all_news

class NewsMonitor:
    def __init__(self, tickers, batch_size=10):
        self.tickers = tickers
        self.batch_size = batch_size
        self.fetcher = NewsFetcher(FINNHUB_API_KEY)  # Initialize fetcher here

    async def monitor_news(self):
        async with aiohttp.ClientSession() as session:
            while True:
                cycle_start = datetime.now(pytz.UTC)
                logger.info(f"[Chart] Monitoring {len(self.tickers)} stocks (batch size: {self.batch_size})")
                logger.info("Starting news monitoring system...")
                
                is_market_open = self._is_market_open()
                cycle_sleep = 10 if is_market_open else 300  # 10s during market hours, 5min otherwise
                
                total_items = 0
                for i in range(0, len(self.tickers), self.batch_size):
                    batch = self.tickers[i:i + self.batch_size]
                    logger.debug(f"Processing batch {batch}")
                    news_items = await self.fetcher.fetch_batch(batch, session)
                    total_items += len(news_items)
                    logger.info(f"[Info] Processed {len(news_items)} items for batch {batch}")
                    await asyncio.sleep(1)  # 1-second post-batch sleep

                cycle_end = datetime.now(pytz.UTC)
                cycle_time = (cycle_end - cycle_start).total_seconds()
                logger.info(f"End of a cycle. Cycle took {cycle_time:.2f} seconds. Total new items: {total_items}")
                
                await asyncio.sleep(cycle_sleep)  # Dynamic sleep

    def _is_market_open(self):
        us_tz = pytz.timezone('US/Eastern')
        now = datetime.now(us_tz)
        us_holidays = holidays.US(years=now.year)
        return now.weekday() <= 4 and 9.5 <= now.hour + now.minute/60 < 16 and now.date() not in us_holidays


    def fetch_batch(self, batch):
        # Placeholder: Replace with async fetch_batch from NewsFetcher
        results = {}
        for ticker in batch:
            try:
                result = {"items": [f"Mock news for {ticker}"], "request": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC'), "response": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}
                results[ticker] = result
                logger.info(f"[API] Fetched {len(result['items'])} items for {ticker}")
            except Exception as e:
                logger.error(f"[Error] Failed to fetch {ticker}: {e}")
                results[ticker] = {"items": [], "request": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC'), "response": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}
        time.sleep(1)  # 1-second sleep post-batch
        return results

    def process_batch(self, results):
        total_items = sum(len(v["items"]) for v in results.values())
        return total_items

class HealthChecker:
    def __init__(self, fetcher, processor, tickers):
        self.fetcher = fetcher
        self.processor = processor
        self.tickers = tickers
        self.incidents = []

    async def log_health_check(self, message):
        try:
            health_check_logger.info(message)
        except Exception as e:
            logger.error(f"[Error] Failed to log to HealthCheckLogger: {e}")
            self.incidents.append(f"Failed to log to health_check.log: {str(e)}")

    async def check_missing_news(self, session, is_market_open):
        self.incidents = []
        hkt_tz = pytz.timezone('Asia/Hong_Kong')
        now_hkt = datetime.now(hkt_tz).strftime('%Y-%m-%d %H:%M HKT')
        logger.info(f"Checking missing news at {now_hkt}...")
        if not self.processor.latest_ticker_timestamps:
            logger.warning(f"News database metadata is empty at {now_hkt}.")
            self.incidents.append("News database metadata is empty.")
            await self.log_health_check(f"News database metadata is empty at {now_hkt}")
            return
    
        missing_news_alerts = []
        for ticker in self.tickers:  # Check all tickers
            latest_time = self.processor.latest_ticker_timestamps.get(ticker, 0)
            if latest_time == 0:
                logger.warning(f"[Warning] No news timestamp for {ticker}.")
                missing_news_alerts.append(f"- **{ticker}**: No news timestamp recorded.")
                continue
            latest_dt = datetime.fromtimestamp(latest_time, tz=pytz.UTC).astimezone(hkt_tz)
            if (datetime.now(hkt_tz) - latest_dt).total_seconds() > 6 * 3600:
                missing_news_alerts.append(f"- **{ticker}**: Latest news is >6h old ({latest_dt.strftime('%Y-%m-%d %H:%M HKT')})")
                logger.warning(f"[Warning] Missing news for {ticker}: Latest news is >6h old.")
    
        if missing_news_alerts:
            logger.info("Missing news detected for some tickers.")
            self.incidents.append("Missing news detected for some tickers.")
            await self.log_health_check(f"Missing news detected at {now_hkt}: {', '.join(missing_news_alerts)}")
        else:
            logger.info("No missing news detected for any ticker.")
            await self.log_health_check(f"No missing news detected at {now_hkt}")

    async def health_check(self, session):
        self.incidents = []
        hkt_tz = pytz.timezone('Asia/Hong_Kong')
        now_hkt = datetime.now(hkt_tz).strftime('%Y-%m-%d %H:%M HKT')
        sample_tickers = self.tickers[:3]
        raw_news = []
        logger.info(f"Starting health check news fetching for {len(sample_tickers)} tickers at {now_hkt}...")
        try:
            news_for_batch = await self.fetcher.fetch_batch(sample_tickers, session)
            raw_news.extend(news_for_batch)
        except Exception as e:
            logger.error(f"[Error] Failed to fetch news for health check: {e}")
            self.incidents.append(f"Failed to fetch news for health check: {str(e)}")
        logger.info(f"Finished fetching news for health check. Total items fetched: {len(raw_news)}")

        if not raw_news:
            logger.info(f"[Info] Health check: No news returned for test tickers at {now_hkt}.")
            await self.log_health_check(f"No news returned for test tickers at {now_hkt}")
            return True

        match_count = 0
        for item in raw_news:
            try:
                if self.processor._news_exists(item):
                    match_count += 1
                else:
                    self.incidents.append(f"News item not found on disk: {item['id']}")
            except Exception as e:
                logger.error(f"[Error] Error checking news item {item.get('id', 'N/A')} during health check: {e}")
                self.incidents.append(f"Error checking news item {item.get('id', 'N/A')}: {str(e)}")

        match_percentage = (match_count / len(raw_news)) * 100 if raw_news else 100
        logger.info(f"Health check: {match_percentage:.2f}% match at {now_hkt}")
        await self.log_health_check(f"Health check: {match_percentage:.2f}% match at {now_hkt}")
        return match_percentage >= 90

class Scheduler:
    def __init__(self, tickers, batch_size, fetcher, processor, health_checker):
        self.tickers = tickers
        self.batch_size = batch_size
        self.fetcher = fetcher
        self.processor = processor
        self.health_checker = health_checker
        self.last_daily_clear_date = None
        self.last_health_check = datetime.min.replace(tzinfo=pytz.UTC)

    def _is_market_open(self):
        us_tz = pytz.timezone('US/Eastern')
        now = datetime.now(us_tz)
        us_holidays = holidays.US(years=now.year)
        return now.weekday() <= 4 and 9.5 <= now.hour + now.minute/60 < 16 and now.date() not in us_holidays

    def _is_friday_6pm(self):
        us_tz = pytz.timezone('US/Eastern')
        now = datetime.now(us_tz)
        return now.weekday() == 4 and now.hour == 18 and now.minute == 0

    async def clear_storage(self, now_et):
        hkt_tz = pytz.timezone('Asia/Hong_Kong')
        now_hkt = datetime.now(hkt_tz).strftime('%Y-%m-%d %H:%M HKT')
        logger.info(f"[Clear] Clearing news data and metadata for {now_hkt}...")
        try:
            if os.path.exists(self.processor.storage_dir):
                shutil.rmtree(self.processor.storage_dir)
                logger.info(f"Removed directory: {self.processor.storage_dir}")
            os.makedirs(self.processor.storage_dir, exist_ok=True)
            self.processor.latest_ticker_timestamps = {}
            self.processor._save_metadata()
            logger.info("Clear completed.")
            self.last_daily_clear_date = now_et.date()
        except Exception as e:
            logger.error(f"[Error] Failed to clear storage: {e}")
            self.health_checker.incidents.append(f"Clear storage error: {str(e)}")

    async def run(self):
        logger.info("Starting news monitoring system...")
        us_tz = pytz.timezone('US/Eastern')
        hkt_tz = pytz.timezone('Asia/Hong_Kong')
    
        async with aiohttp.ClientSession() as session:
            while True:
                try:
                    now_utc = datetime.now(pytz.UTC)
                    now_et = now_utc.astimezone(us_tz)
                    now_hkt = now_utc.astimezone(hkt_tz)
                    logger.debug(f"Current ET: {now_et}, Current HKT: {now_hkt}")
    
                    if self._is_friday_6pm():
                        await self.clear_storage(now_et)
                        await asyncio.sleep(60)
                        continue
    
                    if not self._is_market_open() and now_et.hour >= 16 and (self.last_daily_clear_date is None or self.last_daily_clear_date != now_et.date()):
                        await self.clear_storage(now_et)
                        await asyncio.sleep(60)
                        continue
    
                    is_market_open = self._is_market_open()
                    cycle_sleep = 10 if is_market_open else 300  # 10s during market hours, 5min otherwise
                    start_cycle_time = datetime.now(pytz.UTC)
                    batch_times = [0, 2, 4, 6]
    
                    total_new = 0
                    for i, batch_start_offset in enumerate(batch_times):
                        batch = self.tickers[i * self.batch_size:(i + 1) * self.batch_size]
                        target_start = start_cycle_time + timedelta(seconds=batch_start_offset)
                        while datetime.now(pytz.UTC) < target_start:
                            await asyncio.sleep(0.001)
                        batch_start_time = datetime.now(pytz.UTC)
                        logger.debug(f"Starting batch {i+1} for tickers {batch} at {batch_start_time}")
                        try:
                            news_items = await self.fetcher.fetch_batch(batch, session)
                            logger.debug(f"Fetched {len(news_items)} items for batch {batch}")
                            total_new += await self.processor.process_batch(news_items, session)
                        except Exception as e:
                            logger.error(f"[Error] Failed to process batch {batch}: {e}")
                            self.health_checker.incidents.append(f"Batch processing error: {str(e)}")
                        
                        batch_end_time = datetime.now(pytz.UTC)
                        logger.info(f"End of batch {i+1}")
                        batch_duration = (batch_end_time - batch_start_time).total_seconds()
                        sleep_for_pacing = max(0.0, 1.0 - batch_duration)
                        if sleep_for_pacing > 0:
                            await asyncio.sleep(sleep_for_pacing)
    
                    cycle_end_time = datetime.now(pytz.UTC)
                    cycle_duration = (cycle_end_time - start_cycle_time).total_seconds()
                    logger.info(f"End of a cycle. Cycle took {cycle_duration:.2f} seconds. Total new items: {total_new}")
    
                    if self._is_health_check_time(now_hkt, is_market_open):
                        now_hkt_str = now_hkt.strftime('%Y-%m-%d %H:%M HKT')
                        logger.info(f"Running missing news check at {now_hkt_str}...")
                        try:
                            await self.health_checker.check_missing_news(session, is_market_open)
                        except Exception as e:
                            logger.error(f"[Error] Failed to check missing news: {e}")
                        logger.info(f"Running health check at {now_hkt_str}...")
                        try:
                            await self.health_checker.health_check(session)
                        except Exception as e:
                            logger.error(f"[Error] Health check error at {now_hkt_str}: {e}")
                        self.last_health_check = now_utc
    
                    if cycle_duration < cycle_sleep:
                        logger.info(f"Sleeping for {cycle_sleep - cycle_duration:.2f} seconds after cycle")
                        await asyncio.sleep(cycle_sleep - cycle_duration)
                except Exception as e:
                    logger.error(f"[Critical] Scheduler loop crashed: {e}")
                    await asyncio.sleep(60)

    def _is_health_check_time(self, now_hkt, is_market_open):
        minute = now_hkt.minute
        hour = now_hkt.hour
        if is_market_open:  # 21:30 HKT–04:00 HKT
            return minute in (0, 30)  # Every 30 minutes (e.g., 22:00, 22:30)
        else:  # U.S. off-market hours
            return minute == 0  # Every 60 minutes (e.g., 07:00, 08:00)

class NewsProcessor:
    def __init__(self, storage_dir, discord_webhook, discord_alerts):
        self.storage_dir = storage_dir
        self.discord_webhook = discord_webhook
        self.discord_alerts = discord_alerts
        self.discord_enabled = bool(discord_webhook)
        os.makedirs(storage_dir, exist_ok=True)
        self.latest_ticker_timestamps = self._load_metadata()
        logger.debug(f"NewsProcessor initialized: storage_dir={storage_dir}")

    def _load_metadata(self):
        metadata_path = os.path.join(self.storage_dir, "latest_news_timestamps.json")
        if os.path.exists(metadata_path):
            with open(metadata_path, 'r') as f:
                return json.load(f)
        return {}

    def _save_metadata(self):
        metadata_path = os.path.join(self.storage_dir, "latest_news_timestamps.json")
        with open(metadata_path, 'w') as f:
            json.dump(self.latest_ticker_timestamps, f)

    def _generate_id(self, item):
        if 'datetime' not in item or not isinstance(item['datetime'], (int, float)):
            ts_str = datetime.now().strftime('%Y%m%d%H%M')
        else:
            ts_str = datetime.fromtimestamp(item['datetime'], tz=pytz.UTC).strftime('%Y%m%d%H%M')
        ticker_val = item.get('ticker', 'UNKNOWN_TICKER')
        headline_val = item.get('headline', 'UNKNOWN_HEADLINE').encode()
        return f"{ticker_val}_{ts_str}_{hashlib.md5(headline_val).hexdigest()[:6]}"

    def _get_news_filepath(self, item):
        finnhub_id = item.get('id')
        if not finnhub_id:
            logger.warning(f"Finnhub ID missing for item {item.get('headline', 'N/A')}, using generated ID as fallback")
            finnhub_id = self._generate_id(item)
        timestamp = item.get('datetime', 0)
        if not isinstance(timestamp, (int, float)):
            date_str = datetime.now().strftime('%Y-%m-%d')
        else:
            date_str = datetime.fromtimestamp(timestamp, tz=pytz.UTC).strftime('%Y-%m-%d')
        ticker = item.get('ticker', 'UNKNOWN_TICKER')
        date_dir = os.path.join(self.storage_dir, date_str)
        ticker_dir = os.path.join(date_dir, ticker)
        os.makedirs(ticker_dir, exist_ok=True)
        return os.path.join(ticker_dir, f"{finnhub_id}.json")

    async def process_batch(self, news_items, session):
        logger.info(f"Processing batch with {len(news_items)} items")
        total_new = 0
        hkt_tz = pytz.timezone('Asia/Hong_Kong')
        for item in news_items:
            ticker = item.get('ticker')
            if not ticker:
                logger.warning(f"Skipping news item with no ticker: {item.get('headline', 'N/A')}")
                continue
            timestamp = item.get('datetime', 0)
            if not isinstance(timestamp, (int, float)):
                logger.warning(f"Skipping news item for {ticker} with invalid timestamp: {item.get('headline', 'N/A')}")
                continue

            # Generate or validate ID
            item_id = item.get('id')
            if not item_id:
                item_id = self._generate_id(item)
                item['id'] = item_id
                logger.info(f"Generated ID {item_id} for news item {item.get('headline', 'N/A')}")

            # Standardize JSON structure
            standardized_item = {
                'id': item_id,
                'ticker': ticker,
                'headline': item.get('headline', 'No headline'),
                'datetime': timestamp,
                'url': item.get('url', ''),
                'summary': item.get('summary', ''),
                'source': item.get('source', 'Unknown')
            }

            # Check if new item
            if ticker not in self.latest_ticker_timestamps or timestamp > self.latest_ticker_timestamps.get(ticker, 0):
                self.latest_ticker_timestamps[ticker] = timestamp
                total_new += 1
                logger.info(f"New news for {ticker}: {standardized_item['headline']}")

                # Save to disk with standardized structure
                storage_path = self._get_news_filepath(standardized_item)
                try:
                    with open(storage_path, 'w', encoding='utf-8') as f:
                        json.dump(standardized_item, f, ensure_ascii=False, indent=2)
                    logger.debug(f"Saved news item {finnhub_id} for {ticker} to {storage_path}")
                except Exception as e:
                    logger.error(f"Failed to save news item {finnhub_id} for {ticker}: {e}")
                    continue

                # Send Discord alert
                if self.discord_enabled and self.discord_alerts:
                    hkt_time = datetime.fromtimestamp(timestamp, tz=pytz.UTC).astimezone(hkt_tz).strftime('%Y-%m-%d %H:%M HKT')
                    message = (
                        f"**{ticker} News**:\n"
                        f"**Title**: {standardized_item['headline']}\n"
                        f"**Time**: {hkt_time}\n"
                        f"**URL**: {standardized_item['url']}\n"
                        f"**Summary**: {standardized_item['summary']}\n"
                        f"**Source**: {standardized_item['source']}"
                    )
                    payload = {"content": message}
                    try:
                        async with session.post(self.discord_webhook, json=payload) as response:
                            if response.status == 204:
                                discord_logger.info(f"Posted to Discord: {item_id}", extra={"news_id": item_id, "discord_message": message})
                            else:
                                logger.error(f"Failed to post to Discord: HTTP {response.status}")
                    except Exception as e:
                        logger.error(f"Error posting to Discord: {e}")

        self._save_metadata()
        logger.debug(f"Batch processed, total new items: {total_new}")
        return total_new

    async def _send_discord_alert(self, item, session):
        # This method is now handled within process_batch for consistency
        pass

    def _news_exists(self, item):
        filepath = self._get_news_filepath(item)
        return os.path.exists(filepath)

In [ ]:
async def main():
    fetcher = NewsFetcher(FINNHUB_API_KEY)
    processor = NewsProcessor(NEWS_STORAGE_DIR, DISCORD_WEBHOOK, DISCORD_ALERTS)
    health_checker = HealthChecker(fetcher, processor, TICKERS)
    scheduler = Scheduler(TICKERS, BATCH_SIZE, fetcher, processor, health_checker)
    await scheduler.run()

if __name__ == "__main__":
    asyncio.run(main())